In [33]:
import pandas as pd
import requests
import folium

# API URL
api_url = "https://api.eia.gov/v2/electricity/operating-generator-capacity/data/"
api_key = "Kek68ciZeynS6IPayU8d4ePB2rOmuBQZJCYZbdx5"  

# API parameters
params = {
    'api_key': api_key,
    'frequency': 'monthly', 
    'sort[0][direction]': 'asc',
    'offset': 0,
    'length': 5000
}

# API request
response = requests.get(api_url, params=params)

# Check the status code of the response
if response.status_code == 200:
    data = response.json()

    # Convert the data to a DataFrame
    df = pd.DataFrame(data['response']['data'])

    # Convert 'longitude' and 'latitude' to numeric
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

    # Modify the longitude values to be negative
    df['longitude'] = df['longitude'].apply(lambda x: -abs(x) if x > 0 else x)

    # Define your grouping logic
    grouping_logic = {
        'BIT': 'Coal', 'BLQ': 'Coal', 'LIG': 'Coal', 'PC': 'Coal', 'RC': 'Coal',
        'RFO': 'Coal', 'SUB': 'Coal', 'WC': 'Coal',
        'NG': 'Natural Gas', 'BFG': 'Natural Gas', 'OG': 'Natural Gas', 'LFG': 'Natural Gas', 'PG': 'Natural Gas',
        'DFO': 'Oil', 'JF': 'Oil', 'KER': 'Oil', 'PUR': 'Oil', 'WO': 'Oil',
        'GEO': 'Geothermal', 'H2': 'Geothermal',
        'SUN': 'Solar', 'WND': 'Wind', 'WAT': 'Hydro',
        'AB': 'Renewable', 'OBG': 'Renewable', 'OBL': 'Renewable', 'OBS': 'Renewable',
        'SLW': 'Renewable', 'WDL': 'Renewable', 'WDS': 'Renewable',
        'NUC': 'Nuclear', 'OTH': 'Other'
    }

    # Add a new column for your custom groups based on the energy source
    df['custom_group'] = df['energy_source_code'].map(grouping_logic)

    # Group the data by 'custom_group'
    df_grouped = df.groupby(['custom_group', 'plantid'], as_index=False).agg({
        'net-summer-capacity-mw': 'sum',
        'latitude': 'first',
        'longitude': 'first',
        'entityName': 'first',
        'plantName': 'first',
        'stateName': 'first'
    })

    # Create a base map
    base_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

    # Create a feature group for each custom group
    for group_name in df_grouped['custom_group'].unique():
        feature_group = folium.FeatureGroup(name=group_name)
        base_map.add_child(feature_group)

        # Filter the DataFrame for the current group
        group_df = df_grouped[df_grouped['custom_group'] == group_name]

        # Add markers to the feature group
        for index, row in group_df.iterrows():
            if pd.notna(row['latitude']) and pd.notna(row['longitude']):
                folium.Marker(
                    location=[row['latitude'], row['longitude']],
                    popup=(
                        f"<table>"
                        f"<tr><th>Plant Name</th><td>{row['plantName']}</td></tr>"
                        f"<tr><th>Utility</th><td>{row['entityName']}</td></tr>"
                        f"<tr><th>State</th><td>{row['stateName']}</td></tr>"
                        f"<tr><th>Capacity</th><td>{row['net-summer-capacity-mw']} MW</td></tr>"
                        f"<tr><th>Plant ID</th><td>{row['plantid']}</td></tr>"
                        f"</table>"
                    ),
                    tooltip=row['plantName'],
                    icon=folium.Icon(color=color_map.get(group_name, 'gray'), icon='info-sign')
                ).add_to(feature_group)

    # Add LayerControl to toggle the layers
    folium.LayerControl(collapsed=False).add_to(base_map)

    # Save the map with all layers
    map_filename = "all_energy_types_locations_map.html"
    base_map.save(map_filename)
    print(f"Map with grouped energy types saved as {map_filename}")

else:
    print(f"Failed to retrieve data: {response.status_code}")

Map with grouped energy types saved as all_energy_types_locations_map.html
